In [2]:
import pandas as pd

In [3]:
binding_scores = pd.read_csv('drug_gene_binding_scores_matrix.csv')
reg_scores_up = pd.read_csv('disease_drug_interactions_up.csv')
reg_scores_down = pd.read_csv('disease_drug_interactions_down.csv')

In [4]:
dis_gene = {}
for col in reg_scores_up.columns:  
    if col[:4] == 'DOID':
        values = reg_scores_up[col].values
        not_nan_values = values[~pd.isna(values)]
        not_nan_index = [i for i, v in enumerate(values) if not pd.isna(v)]
        print(f"{col},Indices of Non-NaN values: {not_nan_index}")
        dis_gene[col] = {'up':not_nan_index}


DOID:1240,Indices of Non-NaN values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
DOID:6432,Indices of Non-NaN values: [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
DOID:3498,Indices of Non-NaN values: [23, 73, 74, 75, 76, 97, 108, 112, 114, 126, 145, 174, 196, 229, 233, 243, 251, 252, 253, 254]
DOID:2154,Indices of Non-NaN values: [24, 40, 41, 81, 82, 83, 84, 99, 109, 127, 194, 196, 197, 207, 213, 221, 236, 244, 258, 259]
DOID:2394,Indices of Non-NaN values: [24, 43, 54, 58, 59, 60, 61, 87, 123, 124, 160, 161, 165, 192, 248, 260, 261, 262, 263, 264]
DOID:0050589,Indices of Non-NaN values: [25, 35, 47, 48, 49, 50, 51, 52, 53, 54, 87, 88, 102, 119, 138, 226, 241, 268, 270, 271]
DOID:971,Indices of Non-NaN values: [25, 72, 81, 110, 123, 165, 170, 172, 173, 182, 192, 205, 206, 214, 240, 255, 272, 273, 274, 275]
DOID:0050742,Indices of Non-NaN values: [27, 32, 53, 59, 63, 68, 69, 70, 71, 91, 94, 95, 132, 188, 224, 227, 228, 277, 

In [5]:
rm_cols = []
for col in reg_scores_up.columns:
    if col not in binding_scores.columns:
        rm_cols.append(col)
print(f"Columns to be removed: {rm_cols}")
reg_scores_up.drop(columns=rm_cols, inplace=True)

Columns to be removed: ['DOID:1240', 'DOID:6432', 'Cetuximab', 'Arc (nsc 188491)', 'Pertuzumab', 'Tamoxifen', 'Casiigly', 'Rosiglitazone', 'Interferon gamma-1b', 'Bicalutamide', 'Trovafloxacin', 'Clinafloxacin', 'Pbde 85', 'Oprelvekin', 'Etanercept', 'Lapatinib', '4-hydroxynonenal', 'Nvp-tnks656 (tankyrase inhibitor)', 'Rituximab', 'DOID:3498', 'DOID:2154', 'DOID:2394', 'Pd173074', 'Interferon beta-1b', 'Interferon beta-1a', 'DOID:0050589', 'DOID:971', 'DOID:0050742', 'Laccaic acid', 'DOID:5082', 'DOID:3910', 'DOID:381', 'Natural alpha interferon', 'DOID:7147', 'DOID:9008', 'DOID:5409', 'Bleomycin', 'DOID:4556', 'DOID:419', 'DOID:13949', 'DOID:9256', 'DOID:4450']


In [6]:
df1 = binding_scores.set_index('Gene')
df2 = reg_scores_up.set_index('Gene')
df1.fillna(1, inplace=True)
df2.fillna(0, inplace=True)
product = df1.mul(df2, fill_value=0)
product.reset_index(inplace=True)

In [7]:
product.head()

,Gene,"1,2,4-benzenetriol","1,25 dihydroxyvitamin d","2,2',4,4',5,5'-hexachlorobiphenyl (pcb-153)","2,3,7,8-tetrachlorodibenzo-p-dioxin","2,3,7,8-tetrachlorodibenzo-p-dioxin (tcdd)","2,3-dimethoxy-1,4-naphtoquinone (dmnq)","3,3',4,4'-tetrachlorobiphenyl",4-hydroxytamoxifen,Abiraterone,...,Torcetrapib,Trastuzumab,Tretinoin,Triiodothyronine-[13c6] hydrochloride (t3 thyronine),Ubiquinol,Vanadium pentoxide,Vemurafenib,Vx,Y15,Zinc acetate
0,A2M,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
1,ACADS,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,ACO1,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,6.08826,...,0.000000,0.0,0.0,0.000000,0.0,0.0,202.704342,0.0,0.0,0.000000
3,ACTA1,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
4,ACTB,2522.41997,0.0,884.704055,0.0,28868.570262,0.0,0.0,0.0,10.43850,...,26.673648,0.0,0.0,16.337568,0.0,0.0,0.000000,0.0,0.0,6361.900188


In [8]:
for col in reg_scores_down.columns:  
    if col[:4] == 'DOID':
        values = reg_scores_down[col].values
        not_nan_values = values[~pd.isna(values)]
        not_nan_index = [i for i, v in enumerate(values) if not pd.isna(v)]
        dis_gene[col]['down'] = not_nan_index


In [9]:
rm_cols = []
for col in reg_scores_down.columns:
    if col not in binding_scores.columns:
        rm_cols.append(col)
print(f"Columns to be removed: {rm_cols}")
reg_scores_down.drop(columns=rm_cols, inplace=True)

Columns to be removed: ['DOID:1240', 'DOID:6432', 'Etanercept', 'Lapatinib', 'Interferon beta-1a', 'Laccaic acid', 'Pd173074', 'Oprelvekin', 'Rosiglitazone', 'Pbde 85', 'Interferon gamma-1b', 'Tamoxifen', 'DOID:0050589', 'DOID:9256', '4-hydroxynonenal', 'Interferon beta-1b', 'Natural alpha interferon', 'DOID:971', 'Cetuximab', 'Rituximab', 'DOID:2154', 'DOID:2394', 'Pertuzumab', 'DOID:13949', 'Bicalutamide', 'DOID:419', 'DOID:381', 'Bleomycin', 'DOID:3498', 'Arc (nsc 188491)', 'DOID:7147', 'Clinafloxacin', 'Nvp-tnks656 (tankyrase inhibitor)', 'DOID:3910', 'DOID:5409', 'Trovafloxacin', 'Casiigly', 'DOID:5082', 'DOID:4450', 'DOID:4556', 'DOID:0050742', 'DOID:9008']


In [10]:
df2 = reg_scores_down.set_index('Gene')
df2.fillna(0, inplace=True)
product2 = df1.mul(df2, fill_value=0)
product2.reset_index(inplace=True)

In [11]:
product2.tail()

,Gene,"1,2,4-benzenetriol","1,25 dihydroxyvitamin d","2,2',4,4',5,5'-hexachlorobiphenyl (pcb-153)","2,3,7,8-tetrachlorodibenzo-p-dioxin","2,3,7,8-tetrachlorodibenzo-p-dioxin (tcdd)","2,3-dimethoxy-1,4-naphtoquinone (dmnq)","3,3',4,4'-tetrachlorobiphenyl",4-hydroxytamoxifen,Abiraterone,...,Torcetrapib,Trastuzumab,Tretinoin,Triiodothyronine-[13c6] hydrochloride (t3 thyronine),Ubiquinol,Vanadium pentoxide,Vemurafenib,Vx,Y15,Zinc acetate
519,YWHAE,0.0,0.00000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0
520,ZFP36,0.0,0.00000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,-3766.419218,-5803.11888,0.0,-164.583519,0.0,0.0
521,ZG16,0.0,0.00000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0
522,ZNF781,0.0,0.00000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0
523,ZWINT,0.0,-250.34994,-703.01595,0.0,0.0,0.0,-12769.153096,0.0,0.0,...,-31.042764,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0


In [ ]:
import os
os.makedirs('pred_drugs', exist_ok=True)
os.makedirs('predictions_csv', exist_ok=True)

In [13]:
def sanitize_name(x):
    if isinstance(x, (list, tuple, set)):
        return "_".join(map(str, x))
    if hasattr(x, "tolist"):
        return "_".join(map(str, x.tolist()))
    return str(x).replace(":", "_").replace(" ", "_")


In [17]:
dis_drug_up = {}

for disease in dis_gene.keys():
    disease_name = sanitize_name(disease)
    print(disease_name)
    # IMPORTANT: use loc if indices are gene names
    sub = product.loc[dis_gene[disease]['up'], product.columns[1:]]

    if sub.empty:
        print(f"⚠️ No data for {disease_name}")
        continue

    s = sub.sum(axis=0).dropna()
    s_nonzero = s[s != 0].sort_values(ascending=False)

    out_path = os.path.join('pred_drugs', f"{disease_name}_drugs.txt")
    
    print(s_nonzero)
    with open(out_path, "w") as f:
        f.write("Up reg Drug\tScore\n")
        for drug, score in s_nonzero.items():
            f.write(f"{drug}\t{score}\n")

    dis_drug_up[disease_name] = s_nonzero.to_dict()


DOID_1240
2,3,7,8-tetrachlorodibenzo-p-dioxin (tcdd)              28868.570262
Mercury                                                 27429.011702
Antimony                                                24733.454859
Curcumin                                                22222.185375
Lysophosphatidic acid                                   20113.507910
                                                            ...     
Pimecrolimus                                               16.469311
Triiodothyronine-[13c6] hydrochloride (t3 thyronine)       16.337568
Hodgkin lymphoma                                           15.511391
Testosterone                                                6.447015
Dexamethasone                                               3.696375
Length: 93, dtype: float64
DOID_6432
Mercury         16012.073985
Cisplatin       14917.383997
Chromium        11884.523970
Zinc acetate     9864.462303
Phenol           9734.954695
                    ...     
Fluticasone         

In [18]:
dis_drug_down = {}
for disease in dis_gene.keys():
    disease_name = sanitize_name(disease)
    print(disease_name)
    # IMPORTANT: use loc if indices are gene names
    sub = product2.loc[dis_gene[disease]['down'], product2.columns[1:]]

    if sub.empty:
        print(f"⚠️ No data for {disease_name}")
        continue

    s = sub.sum(axis=0).dropna()
    s_nonzero = s[s != 0].sort_values()

    out_path = os.path.join('pred_drugs', f"{disease_name}_drugs.txt")
    
    print(s_nonzero)
    with open(out_path, "a") as f:
        f.write('\n'+'-'*20 + "\n")
        f.write("Down reg Drug\tScore\n")
        for drug, score in s_nonzero.items():
            f.write(f"{drug}\t{score}\n")

    dis_drug_down[disease_name]=s_nonzero.to_dict()


DOID_1240
Gefitinib                                    -102625.107372
Hydroxyzine                                   -40288.811645
Bisphenol a                                   -31437.954062
2,3,7,8-tetrachlorodibenzo-p-dioxin (tcdd)    -26896.510025
Cisplatin                                     -26153.092244
                                                  ...      
Phorbol 12-myristate 13-acetate (pma)            -22.259024
Calcitriol                                       -14.143248
Testosterone                                      -9.841150
Torcetrapib                                       -7.763072
Fluticasone                                       -0.999064
Length: 92, dtype: float64
DOID_6432
Vanadium pentoxide                           -15072.591828
Heregulin                                    -10411.993697
Mycophenolic acid                             -6077.692672
Mercury                                       -5896.092872
Temozolomide                                  -4098.70100

### Finding drugs common for both up and down reg

In [19]:
rows = []

for disease in dis_drug_up.keys():
    drugs_up = dis_drug_up[disease]
    drugs_down = dis_drug_down.get(disease, {})

    common = {
        k: drugs_up[k] + abs(drugs_down[k])
        for k in drugs_up.keys() & drugs_down.keys()
        if drugs_up[k] + abs(drugs_down[k]) <= 10000
    }

    if not common:
        print(disease, "→ no common found")
        continue

    # sort by combined score (descending)
    sorted_common = sorted(common.items(), key=lambda x: x[1], reverse=True)

    for rank, (drug, score) in enumerate(sorted_common, start=1):
        rows.append({
            "Disease": disease,
            "Drug": drug,
            "Combined_Score": score
        })

final_df = pd.DataFrame(rows)


In [34]:
final_df.head()

,Disease,Drug,Combined_Score
0,DOID_1240,"Puromycin, ec50, 5 d",7131.331564
1,DOID_1240,Propofol,6621.403639
2,DOID_1240,Motexafin gadolinium,4552.401823
3,DOID_1240,Insulin,4421.479649
4,DOID_1240,Letrozole,4129.512797


In [ ]:
import re

def normalize_drug_name(name):
    if pd.isna(name):
        return name
    # remove anything inside brackets
    name = re.sub(r"\s*[\(\[\{].*?[\)\]\}]", "", name)
    return name.strip()
final_df['Drug'] = final_df['Drug'].apply(normalize_drug_name)

# sort first (important!)
final_df = final_df.sort_values(
    ['Disease', 'Combined_Score'],
    ascending=[True, False]
)

# group and collect drugs into a list
result_df = (
    final_df
    .groupby('Disease')['Drug']
    .apply(list)
    .reset_index(name='Drugs')
)
result_df
result_df.to_csv('predictions_csv/final_drug_predictions_grouped.csv', index=False)

### Combined Score of all the drugs

In [20]:
rows = []

for disease in dis_drug_up.keys():
    drugs_up = dis_drug_up[disease]
    drugs_down = dis_drug_down.get(disease, {})

    scores = {}

    # ---------- common drugs ----------
    for drug in drugs_up.keys() & drugs_down.keys():
        combined_score = drugs_up[drug] + abs(drugs_down[drug])
        if combined_score < 7000:
            scores[drug] = combined_score

    # ---------- up-only drugs ----------
    for drug in drugs_up.keys() - drugs_down.keys():
        score = abs(drugs_up[drug])
        if score < 7000:
            scores[drug] = score

    # ---------- down-only drugs ----------
    for drug in drugs_down.keys() - drugs_up.keys():
        score = abs(drugs_down[drug])
        if score < 7000:
            scores[drug] = score

    if not scores:
        print(disease, "→ no drugs found")
        continue

    # sort by score (descending)
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    for rank, (drug, score) in enumerate(sorted_scores, start=1):
        rows.append({
            "Disease": disease,
            "Drug": drug,
            "Score": score,
            "Rank": rank
        })
        # if rank>=30:
        #     break

final_df = pd.DataFrame(rows)


In [21]:
final_df

,Disease,Drug,Score,Rank
0,DOID_1240,Propofol,6621.403639,1
1,DOID_1240,Azacitidine,6191.337898,2
2,DOID_1240,Mycophenolic acid,5771.535153,3
3,DOID_1240,Phenol,5756.063625,4
4,DOID_1240,Methylphenidate,5261.085275,5
...,...,...,...,...
1922,DOID_4450,Chlorpyrifos,13.453427,87
1923,DOID_4450,Torcetrapib,11.052849,88
1924,DOID_4450,Thapsigargin,9.676805,89
1925,DOID_4450,Testosterone,9.318921,90


In [ ]:
import re

def normalize_drug_name(name):
    if pd.isna(name):
        return name
    # remove anything inside brackets
    name = re.sub(r"\s*[\(\[\{].*?[\)\]\}]", "", name)
    return name.strip()
final_df['Drug'] = final_df['Drug'].apply(normalize_drug_name)

# sort first (important!)
final_df = final_df.sort_values(
    ['Disease', 'Score'],
    ascending=[True, False]
)

# group and collect drugs into a list
result_df = (
    final_df
    .groupby('Disease')['Drug']
    .apply(list)
    .reset_index(name='Drugs')
)
result_df
result_df.to_csv('predictions_csv/final_drug_predictions_grouped_all.csv', index=False)

#### Top 30 only

In [23]:
rows = []

for disease in dis_drug_up.keys():
    drugs_up = dis_drug_up[disease]
    drugs_down = dis_drug_down.get(disease, {})

    scores = {}

    # ---------- common drugs ----------
    for drug in drugs_up.keys() & drugs_down.keys():
        combined_score = drugs_up[drug] + abs(drugs_down[drug])
        if combined_score < 7000:
            scores[drug] = combined_score

    # ---------- up-only drugs ----------
    for drug in drugs_up.keys() - drugs_down.keys():
        score = abs(drugs_up[drug])
        if score < 7000:
            scores[drug] = score

    # ---------- down-only drugs ----------
    for drug in drugs_down.keys() - drugs_up.keys():
        score = abs(drugs_down[drug])
        if score < 7000:
            scores[drug] = score

    if not scores:
        print(disease, "→ no drugs found")
        continue

    # sort by score (descending)
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    for rank, (drug, score) in enumerate(sorted_scores, start=1):
        rows.append({
            "Disease": disease,
            "Drug": drug,
            "Score": score,
            "Rank": rank
        })
        if rank>=30:
            break

final_df = pd.DataFrame(rows)


In [24]:
final_df

,Disease,Drug,Score,Rank
0,DOID_1240,Propofol,6621.403639,1
1,DOID_1240,Azacitidine,6191.337898,2
2,DOID_1240,Mycophenolic acid,5771.535153,3
3,DOID_1240,Phenol,5756.063625,4
4,DOID_1240,Methylphenidate,5261.085275,5
...,...,...,...,...
565,DOID_4450,"1,25 dihydroxyvitamin d",1033.329281,26
566,DOID_4450,Dasatinib,1015.344199,27
567,DOID_4450,"Puromycin, ec50, 3 d",968.975670,28
568,DOID_4450,Sulforaphane,907.503090,29


In [ ]:
import re

def normalize_drug_name(name):
    if pd.isna(name):
        return name
    # remove anything inside brackets
    name = re.sub(r"\s*[\(\[\{].*?[\)\]\}]", "", name)
    return name.strip()
final_df['Drug'] = final_df['Drug'].apply(normalize_drug_name)

# sort first (important!)
final_df = final_df.sort_values(
    ['Disease', 'Score'],
    ascending=[True, False]
)

# group and collect drugs into a list
result_df = (
    final_df
    .groupby('Disease')['Drug']
    .apply(list)
    .reset_index(name='Drugs')
)
result_df
result_df.to_csv('predictions_csv/final_drug_predictions_grouped_all_top30.csv', index=False)